In [1]:
from connect_db import ConnectPostgres
from sqlalchemy import text
import pandas as pd

cp = ConnectPostgres()
engine = cp.load_engine(**cp.db_)

sql = """
select * from tteokbokki.store_review
"""
with engine.connect() as connection:
    df =  pd.read_sql(text(sql), connection)

In [2]:
df.isna().sum()

store_id                         0
user_nickname                    0
user_review_cnt                  0
user_photo_cnt                   0
user_follower_cnt                0
user_photo_path              56937
user_reservation_location    67167
user_feature                     0
user_review                  37656
user_hashtag                     0
user_visit_date                  0
user_visit_cnt                   0
user_verification                0
crawling_datetime                0
dtype: int64

- 4월 7일 이후 리뷰 제거 후, 리뷰와 비교해서 가게 정보에 있는 리뷰 수 업데이트

In [3]:
df.user_visit_date.min(), df.user_visit_date.max()

(datetime.datetime(200, 9, 6, 0, 0), datetime.datetime(2025, 5, 4, 0, 0))

In [4]:
df.query("user_visit_date < @pd.Timestamp('2000-01-01')") # 데이터를 제외하고 범위가 2020년대이므로 2020, 2021, 2022년도 데이터로 간주

,store_id,user_nickname,user_review_cnt,user_photo_cnt,user_follower_cnt,user_photo_path,user_reservation_location,user_feature,user_review,user_hashtag,user_visit_date,user_visit_cnt,user_verification,crawling_datetime
17714,38226902,도로시2672,2729,137,0,None,None,[점심에 방문],좋아요,[],0200-09-06 00:00:00,1,영수증,2025-04-07 00:00:00.000
70631,1154167883,2YE3,7,5,0,[],,[저녁에 방문],"양도 진짜 많고 사장님, 알바생들이 너무 친절해서 좋아요 ㅠㅠ 무엇보다 떡볶이가 존...","[인테리어가 멋져요, 음악이 좋아요, 친절해요, 음식이 맛있어요, 술이 다양해요]",0201-12-20 00:00:00,1,영수증,2025-05-02 23:25:30.445
76667,33771217,jon****,1414,0,0,[],,[점심에 방문],좋아요,[음식이 맛있어요],0202-04-21 00:00:00,1,영수증,2025-05-04 23:57:00.086


In [5]:
df.query("user_nickname.isin(['도로시2672', '2YE3', 'jon****']) and user_visit_date > @pd.Timestamp('2000-01-01')").groupby('user_nickname').user_visit_date.agg(['min', 'max'])

,min,max
user_nickname,,
2YE3,2021-12-30 00:00:00,2021-12-30 00:00:00
jon****,2020-09-12 00:00:00,2024-07-30 00:00:00
도로시2672,2020-12-20 00:00:00,2024-04-21 00:00:00


In [6]:
df.query("user_nickname.isin(['도로시2672', '2YE3', 'jon****'])").groupby('user_nickname').user_visit_date.agg(['min', 'max'])

,min,max
user_nickname,,
2YE3,0201-12-20 00:00:00,2021-12-30 00:00:00
jon****,0202-04-21 00:00:00,2024-07-30 00:00:00
도로시2672,0200-09-06 00:00:00,2024-04-21 00:00:00


In [7]:
df.loc[17569, 'user_visit_date'] = pd.Timestamp('2020-09-06')
df.loc[70545, 'user_visit_date'] = pd.Timestamp('2021-12-20')
df.loc[76667, 'user_visit_date'] = pd.Timestamp('2022-04-21')

In [8]:
df.user_visit_date.min(), df.user_visit_date.max()

(datetime.datetime(200, 9, 6, 0, 0), datetime.datetime(2025, 5, 4, 0, 0))

In [9]:
print(df.shape)
new_df = df.query("user_visit_date < @pd.Timestamp('2025-04-08')").reset_index(drop=True) # 4월 7일까지로 
print(new_df.shape)

(80111, 14)
(79673, 14)


In [10]:
sql = """
select * from tteokbokki.store
"""
with engine.connect() as connection:
    store =  pd.read_sql(text(sql), connection)

In [11]:
compare_cnt = store.filter(['store_id', 'store_visit_review_cnt'])\
                    .merge(new_df.groupby(['store_id']).user_nickname.count().reset_index(name='cnt'),
                            how= 'right',
                            on = ['store_id'])

In [12]:
compare_cnt.query("store_visit_review_cnt > cnt") # 안녕유부와 진심왕돈가스를 제외하고는 크게 차이 안남 => 덮여쓰기여쓰기

,store_id,store_visit_review_cnt,cnt
3,18091308,960.0,959
5,19550724,383.0,381
7,19728386,242.0,241
11,19877681,1569.0,1568
12,19900030,600.0,599
16,20846727,160.0,159
30,33618216,670.0,669
31,33771217,1265.0,1264
41,36831532,391.0,390
49,37699840,668.0,666


In [13]:
compare_cnt.query("store_visit_review_cnt < cnt")

,store_id,store_visit_review_cnt,cnt
0,13547324,1278.0,1292
14,20601524,1022.0,1049
21,20919605,190.0,193
23,30816636,1316.0,1330
33,35092309,1435.0,1454
37,36219909,855.0,866
38,36246944,256.0,260
58,199240131,505.0,508
64,667655112,589.0,595
70,1001854998,1274.0,1275


In [ ]:
# chg_cnt = {i: j for i, j in zip(compare_cnt['store_id'], compare_cnt['cnt'])}
# store['store_visit_review_cnt'] = store.apply(lambda x: chg_cnt[x['store_id']] if x['store_id'] in chg_cnt.keys() else x['store_visit_review_cnt'], axis=1)

In [30]:
from sqlalchemy.sql import update
from sqlalchemy import Table, MetaData

metadata = MetaData()
updated_table = Table('store', metadata, autoload_with=engine)

data_to_update = compare_cnt.query("store_visit_review_cnt != cnt").filter(['store_id', 'cnt']).to_dict('records')

with engine.connect() as connection:
    for row in data_to_update:
        stmt = update(updated_table).where(updated_table.c.store_id == row['store_id']).values(store_visit_review_cnt=row['cnt'])
        connection.execute(stmt)
    connection.commit()

- 의미없는 데이터 확인하기
    - user_review랑 user_hashtag 둘 다 값이 없는 경우 => 방문 횟수를 통해서 정보를 얻을 수 있을 것으로 예상되어 보류

In [31]:
no_review = new_df.query("user_review.isna()") 
no_review_hashtag = no_review.loc[no_review.user_hashtag.map(lambda x: len(x) == 0 or (len(x) == 1 and x[0] == ''))]
print(no_review_hashtag.shape)
no_review_hashtag.head()

(14154, 14)


,store_id,user_nickname,user_review_cnt,user_photo_cnt,user_follower_cnt,user_photo_path,user_reservation_location,user_feature,user_review,user_hashtag,user_visit_date,user_visit_cnt,user_verification,crawling_datetime
1,1982875847,안성민6445,518,19,0,None,None,[],None,[],2021-02-06 00:00:00,1,영수증,2025-04-07
2,1616849156,여행맛집좋아하는,170,45,1,None,None,[],None,[],2020-08-07 00:00:00,1,영수증,2025-04-07
5,1616849156,lee****,788,1,0,None,None,[],None,[],2021-02-06 00:00:00,1,영수증,2025-04-07
24,13578905,Xrp,1687,20,0,None,None,[],None,[],2021-08-29 00:00:00,16,영수증,2025-04-07
38,1704553037,콩순이693,256,6,0,None,,[],None,[],2021-08-27 00:00:00,1,영수증,2025-04-07


- user_hashtag

In [32]:
# [혼, 밥, ///]
hashtag_wrong_split_idx = new_df.loc[new_df.user_hashtag.map(lambda x: False if len(x) == 0 else len(x[0]) == 1)].index
new_df.loc[hashtag_wrong_split_idx, 'user_hashtag'] = new_df.loc[hashtag_wrong_split_idx].user_hashtag.map(lambda x: [''.join(x)])

In [33]:
hashtag_none_idx = new_df.loc[new_df.user_hashtag.map(lambda x: len(x) == 0)].index
new_df.loc[hashtag_none_idx, 'user_hashtag'] = None

In [34]:
# ['']
hashtag_not_null = new_df.query("user_hashtag.notna()")
hashtag_wrong_space = hashtag_not_null.loc[hashtag_not_null.user_hashtag.map(lambda x: len(x) == 1 and x[0] == '')].index
new_df.loc[hashtag_wrong_space, 'user_hashtag'] =  None

- user_feature

In [35]:
feature_not_null = new_df.query("user_feature.notna()")
feature_none_idx = feature_not_null.loc[feature_not_null.user_feature.map(lambda x: len(x) == 0)].index
new_df.loc[feature_none_idx, 'user_feature'] = None

In [36]:
feature_not_null = new_df.query("user_feature.notna()")
feature_wrong_space = feature_not_null.loc[feature_not_null.user_feature.map(lambda x: len(x) == 1 and x[0] == '')].index
new_df.loc[feature_wrong_space, 'user_feature'] =  None

In [37]:
feature_not_null = new_df.query("user_feature.notna()")
new_df.loc[feature_not_null.index, "user_feature"] = feature_not_null.user_feature.map(lambda x: x.remove('') if '' in x else x)

- 펼쳐보기 부분만 제거
    - user_hashtag에서 +n 펼쳐보기가 삽입된 데이터 존재

In [38]:
import re

hashtag_not_null = new_df.query("user_hashtag.notna()")
array_two_idx = hashtag_not_null.loc[hashtag_not_null.user_hashtag.map(lambda x: len(x) == 2)].index
print(len(array_two_idx))
new_df.loc[array_two_idx, 'user_hashtag'] = new_df.loc[array_two_idx].user_hashtag.map(lambda x: x[0] if re.match(r'\+[0-9]\n개의 리뷰가 더 있습니다\n펼쳐보기', x[1]) else x)

9434


In [39]:
new_df.tail()

,store_id,user_nickname,user_review_cnt,user_photo_cnt,user_follower_cnt,user_photo_path,user_reservation_location,user_feature,user_review,user_hashtag,user_visit_date,user_visit_cnt,user_verification,crawling_datetime
79668,1723482207,got****,1246,8,0,[],None,[저녁에 방문],굿,[음식이 맛있어요],2022-06-23 00:00:00,1,영수증,2025-05-04 20:45:48.319702
79669,1723482207,앵버야,43,52,0,[],None,[저녁에 방문],진짜 맛있다........,"[음식이 맛있어요, 인테리어가 멋져요, 혼밥하기 좋아요, 뷰가 좋아요, 매장이 청결해요]",2022-06-13 00:00:00,1,영수증,2025-05-04 20:45:49.920308
79670,1723482207,드래곤섭,1218,169,0,[],None,[저녁에 방문],귯굿,"[양이 많아요, 음식이 맛있어요]",2022-07-14 00:00:00,1,영수증,2025-05-04 20:45:51.523833
79671,1723482207,tingueil,1722,13,0,[],None,[점심에 방문],굿,[음식이 맛있어요],2022-08-27 00:00:00,1,영수증,2025-05-04 20:45:53.097943
79672,1723482207,송데이비드호섭,515,23,3,[],None,[점심에 방문],굿,[음식이 맛있어요],2022-06-06 00:00:00,1,영수증,2025-05-04 20:45:54.699938


In [40]:
cp.save_store_review(new_df)

Pandas DataFrame이 성공적으로 저장되었습니다.
